================================================================================================================


Note that this notebook is just for comparing the speed with using the GPU, DON'T use it to preprocess the data

================================================================================================================

Install ckiptagger for word segmentation and opencc for s2t



In [ ]:
pip install -U ckiptagger[tfgpu,gdown]

In [ ]:
pip install opencc-python-reimplemented

In [ ]:
#mount gdrive and import packages
import pandas as pd
import numpy as np
import re
from opencc import OpenCC
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER

from google.colab import drive
import os
drive.mount('/content/gdrive')


Note that the ckiptagger 'ws' modules requires its pretrained model to segment the sentences, so we have to ensure that we've already obtained its pretrained weights. (If not, use `data_utils.download_data_gdown("./") ` to download the model)

For the purpose of saving time, i already upload those models in our gdrive.

In [ ]:
#solving the environment with ckiptagger
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER
#data_utils.download_data_gdown("./") # gdrive-ckip
#ws = WS("./data")

Load the WS pretrained model


In [ ]:
ws=WS("/content/gdrive/My Drive/data")

Here we use re to compile all the contents, and preserve only Chinese character.

Second, we transform all the Chinese characters from simplified Chinese to traditional Chinese.

And finally we find all the empty valuse and drop its column.

In [ ]:
review=pd.read_json('/content/gdrive/My Drive/Colab Notebooks/Dataset/Original_data/reviews.json')

cc = OpenCC('s2t')
for i in range(len(review)):

  tmp=review['content'][i].replace('\n', '')
  cop=re.compile("[^\u4e00-\u9fa5]")
  review.loc[review.content==review['content'][i],'content']=cop.sub('',tmp)
  review.loc[review.content==review['content'][i],'content']=cc.convert(review['content'][i])
  if review['content'][i].strip()=='':
    review=review.drop(index=i)


After dropping the columns, reindex the dataframe.

In [ ]:
review=review.reset_index(drop=True)

After segmenting the sentence, we can get each word in a sentence separately,


reconstruct them into a sentence using space to separate.





In [ ]:
import time


#to check if gpu is necessary
start_time = time.time()

#segment the corpus
tmp_corpus=ws(review['content'])
cut_corpus=[]
for j in tmp_corpus:
  cut_corpus.append(' '.join(j))

for k in range(len(review)):
  review.loc[review.content==review['content'][k],'content']=cut_corpus[k]

print("My program took",time.time() - start_time, "seconds to run on cpu")


My program took 45.87398886680603 seconds to run on cpu


Save the final result.

In [ ]:
review.to_csv('/content/gdrive/My Drive/Colab Notebooks/reviews_cut.csv',encoding='utf-8',index=False)